# Intro to Quantum Computing with Python and Q\# #

This notebook uses the Python ineroperability for the [Quantum Development Kit](https://github.com/Microsoft/Quantum) to introduce users to Q\# and quantum computing more generally. Part of [this](https://mybuild.techcommunity.microsoft.com/sessions/77163) session at Microsoft Build 2019.

Installation instructions for running this notebook on your machine can be found [here](https://docs.microsoft.com/en-ca/quantum/install-guide/python?view=qsharp-preview).

---

Let's get started by loading the packages we need, including the Q# interoperability package `qsharp`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import qsharp
qsharp.component_versions()

{'iqsharp': LooseVersion ('0.6.1905.301'),
 'Jupyter Core': LooseVersion ('1.1.13141.0'),
 'qsharp': LooseVersion ('0.6.1905.301')}

## What is a Qubit?

In [ ]:
do something with numpy

From a mathematical perspective, a qubit can be represented by a vector of 2 complex numbers like this:

$\vec{x} = \left[\begin{matrix} 0.25 + 0\times i \\0.75 + 0\times i \end{matrix}\right]$

We will often use short hand to label these vectors with a special notation called a ket:

$\left|x\right>$

## Writing Q\# in Python

There are 2 main ways we can use Q# code from Python. The first is we can just directly pass the source to the `qsharp.compile` method:

### Method 1: pass source to compile directly

In [3]:
prepare_qubit = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation PrepareQubit() : Unit {
    using (qubit = Qubit()) {     // We want 1 qubit to use for our task
            DumpMachine();         // Print out what the simulator is keeping a record of
            Reset(qubit);       // We have to return our qubit to the simulator the same way we got it
    }
}
""")

In [91]:
help(prepare_qubit)

Help on QSharpCallable in module qsharp.loader object:

class QSharpCallable(builtins.object)
 |  QSharpCallable(callable_name: str, source: str)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, callable_name: str, source: str)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self) -> str
 |      Return repr(self).
 |  
 |  estimate_resources(self, **kwargs) -> Dict[str, int]
 |  
 |  simulate(self, **kwargs) -> Any
 |      Executes this function or operation on the QuantumSimulator target
 |      machine, returning its output as a Python object.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  

In [92]:
prepare_qubit.simulate()

# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0


()

You can read the above output like the vector we wrote above, where the first column is the index, the second is the real part of the vector at that position, and the second is the complex part of that vector entry.

So the printout
```
0:	1	0
1:	0	0
```
is the same as 
$\left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

In [93]:
prepare_qubit.estimate_resources()

{'CNOT': 0,
 'QubitClifford': 0,
 'R': 0,
 'Measure': 1,
 'T': 0,
 'Depth': 0,
 'Width': 1,
 'BorrowedWidth': 0}

### Method 2: Loading from file

Let's look at the contents of another file in the directory here:

In [26]:
with open('build-demo.qs') as f:
    print(f.read())

// Copyright (c) Sarah Kaiser. All rights reserved.
// Licensed under the MIT License.

namespace Build.Demo {
    open Microsoft.Quantum.Intrinsic;
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Measurement;

/// # Summary
/// Prints out the standard "Hello, world!" message with a
/// bonus favorite number passed as input.
///
/// # Description
/// Takes an `Int` as input and prints the standard test message 
/// as well as what integer the user likes.
///
/// # Input
/// ## number
/// The integer you think is the coolest.
    function HelloWorld (number : Int) : Unit {
        Message($"Hello, world! I like the number {number}.");
    }

/// # Summary
/// Generates a random value from {0,1} by measuring a qubit in 
/// an equal superposition.
///
/// # Description
/// Given a qubit initially in the |0âŸ© state, applies the H operation
/// to that qubit such that it has the state (1/âˆš2) |0âŸ© + (1/âˆš2) |1âŸ©.
/// Measurement of 

We can see that this is a '.qs' file, so it is a Q# source file and had a number of operations and functions we can import:

In [2]:
qsharp.reload()
available_ops = qsharp.get_available_operations_by_namespace()
available_ops['Build.Demo']

['HelloWorld', 'Qrng', 'QrngVerbose']

In [3]:
from Build.Demo import HelloWorld, Qrng, QrngVerbose

In [4]:
?HelloWorld

In [5]:
HelloWorld.simulate(FavoriteNumber=17)

Hello, world! I like the number 17.


()

In [6]:
QrngVerbose.simulate()

Here is what the simulator uses to record a qubit in the 0 state:
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
 
Here is what the simulator uses to record a superposition state:
# wave function for qubits with ids (least to most significant): 0
0:	0.707106781186548	0
1:	0.707106781186548	0


1

In [7]:
[Qrng.simulate() for _ in range(10)]

[1, 0, 1, 0, 0, 1, 0, 0, 1, 0]

---

## Toy quantum algorithim: Deutsch–Jozsa 

TODO: need intro to algorithim here, and graphic

_If I had a function that had one bit input and ouput, how many different options would I have?_

<figure style="text-align: center;">
    <img src="media/twobit.png" width="60%">
    <caption>
      <br>  
        <strong>Diagram of all possible one bit functions</strong>
    </caption>
</figure>

>#### Deutsch–Jozsa Algorithim ####
>**Problem statement:**
>
>* **GIVEN:** A black box quantum operation which takes 1 input bit and produces either a 0 or a 1 as output. We are promised that the box is either _constant_ or _balanced_. 
>					
>* **GOAL:** to determine if the box output is _constant_ or _balanced_ by evaluating sample inputs.

<figure style="text-align: center;">
    <img src="media/twobitDJ.png" width="50%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

In [8]:
available_ops['Build.DeutschJozsa']

['IdOracle',
 'IsOracleBalanced',
 'NotOracle',
 'OneOracle',
 'RunDeutschJozsaAlgorithm',
 'ZeroOracle']

In [9]:
from Build.DeutschJozsa import IsOracleBalanced, NotOracle, ZeroOracle

In [13]:
NotOracle

TypeError: 'NotOracle' object is not callable

In [21]:
check_zero_oracle = qsharp.compile("""
operation IsZeroOracleBalanced(): Bool {Build.DeutschJozsa.IsOracleBalanced(Build.DeutschJozsa.ZeroOracle);}""")

C:/snippet:(2,41): error QS5011: Expecting expression of type Unit. Only expressions of type Unit can be used as expression statements.
C:/snippet:(2,1): error QS6307: Not all code paths return a value.


TypeError: 'NoneType' object is not iterable

In [45]:
from Build.DeutschJozsa import RunDeutschJozsaAlgorithm

In [46]:
RunDeutschJozsaAlgorithm.simulate()

All tests passed!


()

---

## Helpful diagnostics :)

In [5]:
for component, version in sorted(qsharp.component_versions().items(), key=lambda x: x[0]):
    print(f"{component:20}{version}")

Jupyter Core        1.1.13141.0
iqsharp             0.6.1905.301
qsharp              0.6.1905.301


In [6]:
import sys
print(sys.version)

3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]
